# LangServe 학습 노트

1. LangServe란?
- LangChain 팀이 만든 라이브러리
- 목적: LangChain 체인(Chain)이나 Runnable 객체를 REST API로 쉽게 배포
- 내부적으로 FastAPI 와 Pydantic 을 활용
- 장점:
    - 모델/체인을 서버로 띄워서 외부 애플리케이션에서 호출 가능
    - Python뿐 아니라 JavaScript 클라이언트도 제공
    - LangSmith와 통합되어 모니터링/디버깅 가능


설치
- poetry add "langserve[all]"

가상환경에서 실행
- poetry run uvicorn main:app --reload --port 8000


In [ ]:
from fastapi import FastAPI
from langserve import add_routes
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
import os
import uvicorn
from pydantic import BaseModel
from typing import Dict, Any

# .env 파일 로드
#load_dotenv(dotenv_path='../.env')
load_dotenv()

# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:6])

# 입력 스키마 정의
class QuestionInput(BaseModel):
    question: str

# FastAPI 애플리케이션 생성
# FastAPI 앱 객체 생성 → LangServe가 여기에 엔드포인트를 붙임
app = FastAPI(title="LangServe API with .env")

# LLM 모델 생성
llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트 템플릿 설정
# - 입력 변수 {question} 을 받아 답변을 생성하는 템플릿
prompt = PromptTemplate.from_template("질문: {question}\n답변:")

# 체인 생성
# - prompt → llm 으로 연결된 Runnable 체인
chain = prompt | llm

# 방법 1: input_type을 딕셔너리로 설정
add_routes(
    app, 
    chain, 
    path="/chat",
    input_type=Dict[str, Any],  # 딕셔너리 타입으로 변경
    config_keys=["configurable"]
)

# 방법 2: 별도 엔드포인트로 문자열 입력 처리
def process_string_input(input_str: str) -> Dict[str, str]:
    return {"question": input_str}

string_chain = RunnableLambda(process_string_input) | chain

add_routes(
    app,
    string_chain,
    path="/chat_simple",
    input_type=str,  # 문자열 입력
    config_keys=["configurable"]
)

# 방법 3: Pydantic 모델 사용
def process_pydantic_input(input_data: QuestionInput) -> Dict[str, str]:
    return {"question": input_data.question}

pydantic_chain = RunnableLambda(process_pydantic_input) | chain

add_routes(
    app,
    pydantic_chain,
    path="/chat_pydantic",
    input_type=QuestionInput,  # Pydantic 모델 사용
    config_keys=["configurable"]
)

# 헬스체크 엔드포인트
@app.get("/health")
async def health_check():
    return {"status": "healthy"}

# FastAPI 서버 실행
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

gsk_yy


RuntimeError: asyncio.run() cannot be called from a running event loop

📝 RunnableLambda 정리
1. 개념
- LangChain의 LCEL (LangChain Expression Language) 에서 모든 구성 요소는 Runnable 인터페이스를 따릅니다.
- RunnableLambda 는 일반 파이썬 함수(callable) 를 Runnable 로 감싸서, 다른 체인 요소들과 동일하게 연결(|)할 수 있게 해줍니다.

2. 특징
- 임의의 함수 → Runnable 변환
    → 데이터 전처리, 입력 변환, 후처리, 외부 API 호출 등 자유롭게 삽입 가능
- 동기/비동기 모두 지원
    → 함수가 async여도 wrapping 가능
- 체인 조합 가능
    → 프롬프트, 모델, 파서와 동일한 방식으로 파이프라인에 연결

3.주의사항
- RunnableLambda 로 감싼 함수는 입력 인자 1개만 받을 수 있음
    → 여러 인자가 필요하다면 Dict로 묶어서 전달해야 함.
- LangChain의 실행 추적(Tracing)과도 통합되어 디버깅이 쉬움.
